#**1.1 Создание тензоров**

In [13]:
import time
import torch
from tabulate import tabulate

In [21]:
matrix_sizes = [
    (64, 1024, 1024),
    (128, 512, 512),
    (256, 256, 256)
]


def measure_time(func, *args, device='cpu', **kwargs):
    """Данная функция замеряет время работы заданной функции на gpu и cpu"""
    if device == 'cuda' and torch.cuda.is_available():
        torch.cuda.synchronize()
        start = torch.cuda.Event(enable_timing=True)
        end = torch.cuda.Event(enable_timing=True)
        start.record()
        result = func(*args, **kwargs)
        end.record()
        torch.cuda.synchronize()
        return start.elapsed_time(end) / 1000, result
    else:
        start = time.time()
        result = func(*args, **kwargs)
        return time.time() - start, result


operations = {
    'Матричное умножение': lambda a, b: a @ b,
    'Поэлементное сложение': torch.add,
    'Поэлементное умножение': torch.mul,
    'Транспонирование': lambda a: torch.transpose(a, -2, -1),  # Транспонируем последние два измерения
    'Сумма всех элементов': torch.sum
}

all_results = []

for x, y, z in matrix_sizes:

    print(f"\n\nВыполнение операций над матрицами размера {x}x{y}x{z}")

    a_cpu = torch.rand(x, y, z)
    b_cpu = torch.rand(x, y, z)

    if torch.cuda.is_available():
        a_gpu = a_cpu.cuda()
        b_gpu = b_cpu.cuda()
    else:
        a_gpu, b_gpu = None, None

    results = []

    for op_name, op in operations.items():
        row = {'Операция': op_name}

        if op_name in ['Матричное умножение', 'Поэлементное сложение', 'Поэлементное умножение']:
            args_cpu = (a_cpu, b_cpu)
            args_gpu = (a_gpu, b_gpu) if torch.cuda.is_available() else None
        elif op_name == 'Транспонирование':
            args_cpu = (a_cpu,)
            args_gpu = (a_gpu,) if torch.cuda.is_available() else None
        else:
            args_cpu = (a_cpu,)
            args_gpu = (a_gpu,) if torch.cuda.is_available() else None

        cpu_time, _ = measure_time(op, *args_cpu)
        row['CPU (сек)'] = f"{cpu_time:.4f}"

        if torch.cuda.is_available():
            gpu_time, _ = measure_time(op, *args_gpu, device='cuda')
            row['GPU (сек)'] = f"{gpu_time:.4f}"
            row['Ускорение'] = f"{cpu_time/max(gpu_time, 1e-6):.2f}x"
        else:
            row['GPU (сек)'] = "N/A"
            row['Ускорение'] = "N/A"

        results.append(row)

    print(tabulate(results, headers="keys", tablefmt="grid", floatfmt=".4f"))
    all_results.extend(results)



Выполнение операций над матрицами размера 64x1024x1024
+------------------------+-------------+-------------+-------------+
| Операция               |   CPU (сек) |   GPU (сек) | Ускорение   |
+========================+=============+=============+=============+
| Матричное умножение    |      1.1734 |      0.0475 | 24.71x      |
+------------------------+-------------+-------------+-------------+
| Поэлементное сложение  |      0.1940 |      0.0033 | 58.44x      |
+------------------------+-------------+-------------+-------------+
| Поэлементное умножение |      0.1891 |      0.0033 | 57.56x      |
+------------------------+-------------+-------------+-------------+
| Транспонирование       |      0.0000 |      0.0000 | 1.42x       |
+------------------------+-------------+-------------+-------------+
| Сумма всех элементов   |      0.0239 |      0.0011 | 21.80x      |
+------------------------+-------------+-------------+-------------+


Выполнение операций над матрицами размера 12